In [2]:
import os
import requests
import base64
import json
import xmltodict

from datetime import datetime, timedelta
from pathlib import Path
from dotenv import load_dotenv
from urllib.parse import unquote, quote
from typing import Dict, List, Optional, Union

from ebaysdk.trading import Connection
from ebaysdk.exception import ConnectionError
from ebaysdk.finding import Connection

load_dotenv()

CLIENT_ID = os.getenv('EBAY_CLIENT_ID')
CLIENT_SECRET = os.getenv('EBAY_CLIENT_SECRET')
RU_NAME = os.getenv('EBAY_RU_NAME')
ENCODED_AUTH_CODE = os.getenv('EBAY_ENCODED_AUTH_CODE')
# REFRESH_TOKEN = "v^1.1#i^1#f^0#I^3#r^1#p^3#t^Ul4xMF8xMTozNDU2NUMwRUI1QTBBNzEwQTQ3QzU3ODM0MkI2NjM0Ml8xXzEjRV4yNjA="

AUTH_CODE = unquote(ENCODED_AUTH_CODE)

# This is the URL required in order to get a new refresh token (1x every 2 years or 'refresh_token_expires_in': 47304000,). 
# Need to log when this is to anticipate when we need to refresh.
# YOU OPEN THE URL THAT IS CREATED AND GRAB THE TOKEN FROM THE URL IN BROWSER SEARCH BAR - CURRENTLY MANUAL PROCESS
# Code up to only run if refresh token > 1 year to go

# You run the URL to get the AUTH CODE. Only lasts once.
# url = f"https://auth.ebay.com/oauth2/authorize?client_id={CLIENT_ID}&redirect_uri={RU_NAME}&response_type=code&scope=https://api.ebay.com/oauth/api_scope/sell.inventory"
# print(url)

# PASS THE AUTH CODE ... which lasts 1 year ... to the get refresh_token function. T

def get_refresh_token():

    # Step 1: Prepare the request
    token_url = "https://api.ebay.com/identity/v1/oauth2/token"
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": f"Basic {auth_header}"
    }

    data = {
        "grant_type": "authorization_code",
        "code": AUTH_CODE,
        "redirect_uri": RU_NAME
    }

    # Step 2: Make the request
    response = requests.post(token_url, headers=headers, data=data)

    # Step 3: Handle the response
    if response.status_code == 200:
        tokens = response.json()
        print("Access Token:", tokens["access_token"])
        print("Refresh Token:", tokens["refresh_token"])

        print("Expires In:", tokens["expires_in"], "seconds")
        return tokens
    else:
        print("Error:", response.status_code)
        print(response.text)

# This is the function to run ANNUALLY
# tokens = get_refresh_token()

v^1.1#i^1#I^3#p^3#r^1#f^0#t^Ul4xMF81OjZDNjM1N0U3MjExNjlFMkQ5NjFFQzQwMzAzQTUwQTgwXzFfMSNFXjI2MA==


In [30]:
def get_oauth_token(refresh_token):
    """ """

    # Base64 encode the client ID and client secret
    credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
    encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')

    url = 'https://api.ebay.com/identity/v1/oauth2/token'

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': f'Basic {encoded_credentials}'
    }
    data = {
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token,
        'scope': 'https://api.ebay.com/oauth/api_scope/sell.inventory'
    }

    response = requests.post(url, headers=headers, data=data)
    access_data = response.json()

    return access_data['access_token']

def get_active_listings(auth_token):

    endpoint = 'https://api.ebay.com/ws/api.dll'

    headers = {
        'X-EBAY-API-CALL-NAME': 'GetMyeBaySelling',
        'X-EBAY-API-SITEID': '3',  # UK=3, US=0
        'X-EBAY-API-COMPATIBILITY-LEVEL': '1155',
        'X-EBAY-API-IAF-TOKEN': auth_token,
        'Content-Type': 'text/xml'
    }

    # XML request body
    xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <GetMyeBaySellingRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{auth_token}</eBayAuthToken>
        </RequesterCredentials>
        <ActiveList>
            <Include>true</Include>
            <Pagination>
                <EntriesPerPage>200</EntriesPerPage>
                <PageNumber>1</PageNumber>
            </Pagination>
        </ActiveList>
        <DetailLevel>ReturnAll</DetailLevel>
    </GetMyeBaySellingRequest>"""

    try:
        response = requests.post(endpoint, headers=headers, data=xml_request)

        # Convert XML response to dictionary/JSON
        response_dict = xmltodict.parse(response.content)

        # Convert to more manageable JSON structure
        listings = []
        active_list = response_dict.get('GetMyeBaySellingResponse', {}).get('ActiveList', {})
        items = active_list.get('ItemArray', {}).get('Item', [])

        # Handle single item case
        if not isinstance(items, list):
            items = [items]

        for item in items:
            listing = {
                'ItemID': item.get('ItemID'),
                'Title': item.get('Title'),
                'Price': item.get('SellingStatus', {}).get('CurrentPrice', {}).get('@currencyID'),
                'Amount': item.get('SellingStatus', {}).get('CurrentPrice', {}).get('#text'),
                'Quantity': item.get('Quantity'),
                'QuantityAvailable': item.get('QuantityAvailable'),
                'ListingStatus': item.get('SellingStatus', {}).get('ListingStatus'),
                'ViewItemURL': item.get('ListingDetails', {}).get('ViewItemURL'),
                # Add category information
                'PrimaryCategoryID': item.get('PrimaryCategory', {}).get('CategoryID'),
                'PrimaryCategoryName': item.get('PrimaryCategory', {}).get('CategoryName'),
                'SecondaryCategoryID': item.get('SecondaryCategory', {}).get('CategoryID'),
                'SecondaryCategoryName': item.get('SecondaryCategory', {}).get('CategoryName')
            }
            listings.append(listing)

        return listings

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def update_listing(auth_token, item_id, updates):
    endpoint = 'https://api.ebay.com/ws/api.dll'

    headers = {
        'X-EBAY-API-CALL-NAME': 'ReviseItem',
        'X-EBAY-API-SITEID': '3',  # UK=3, US=0
        'X-EBAY-API-COMPATIBILITY-LEVEL': '1155',
        'X-EBAY-API-IAF-TOKEN': auth_token,
        'Content-Type': 'text/xml'
    }

    # Build the update fields XML
    update_fields = ""
    if 'Title' in updates:
        update_fields += f"<Title>{updates['Title']}</Title>"
    if 'Description' in updates:
        update_fields += f"<Description>{updates['Description']}</Description>"
    if 'Price' in updates:
        update_fields += f"<StartPrice>{updates['Price']}</StartPrice>"
    if 'Quantity' in updates:
        update_fields += f"<Quantity>{updates['Quantity']}</Quantity>"

    xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <ReviseItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{auth_token}</eBayAuthToken>
        </RequesterCredentials>
        <Item>
            <ItemID>{item_id}</ItemID>
            {update_fields}
        </Item>
    </ReviseItemRequest>"""

    try:
        response = requests.post(endpoint, headers=headers, data=xml_request)
        response_dict = xmltodict.parse(response.content)
        return response_dict
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

auth_token = get_oauth_token(refresh_token=os.getenv("EBAY_REFRESH_TOKEN"))

listings = get_active_listings(auth_token=auth_token)

if listings:
    for listing in listings:
        print(f"Title: {listing['Title']}")
        print(f"Price: {listing['Amount']} {listing['Price']}")
        print(f"Item ID: {listing['ItemID']}")
        print("-------------------")

listings[0].keys()

Title: Apple iPhone 7 - 32GB - Rose Gold (BT Mobile) A1778 (GSM)
Price: 469.99 GBP
Item ID: 167279800027
-------------------


dict_keys(['ItemID', 'Title', 'Price', 'Amount', 'Quantity', 'QuantityAvailable', 'ListingStatus', 'ViewItemURL', 'PrimaryCategoryID', 'PrimaryCategoryName', 'SecondaryCategoryID', 'SecondaryCategoryName'])

In [31]:
# Examples of Update

updates = {
    'Price': '469.99',
    'Quantity': '1'
}

item_id = "167279800027"

# result = update_listing(access_data['access_token'], item_id, updates)

# result = update_listing(auth_token, item_id, {'Price': '25.99'})

# result = update_listing(auth_token, item_id, {
#     'Title': 'Updated Product Name',
#     'Quantity': '10'
# })

In [ ]:
import requests

EBAY_APP_ID = "YOUR_APP_ID"
EBAY_SITE_ID = "1"  # 0 = US, change for other regions
URL = "https://api.ebay.com/ws/api.dll"

HEADERS = {
    "X-EBAY-API-SITEID": EBAY_SITE_ID,
    "X-EBAY-API-CALL-NAME": "GetCategories",
    "X-EBAY-API-COMPATIBILITY-LEVEL": "967",
    "Content-Type": "text/xml"
}

BODY = """<?xml version="1.0" encoding="utf-8"?>
<GetCategoriesRequest xmlns="urn:ebay:apis:eBLBaseComponents">
    <RequesterCredentials>
        <eBayAuthToken>YOUR_AUTH_TOKEN</eBayAuthToken>
    </RequesterCredentials>
    <DetailLevel>ReturnAll</DetailLevel>
    <CategorySiteID>0</CategorySiteID>
    <ViewAllNodes>true</ViewAllNodes>
</GetCategoriesRequest>"""

response = requests.post(URL, headers=HEADERS, data=BODY)
print(response.text)  # XML response with categories


In [38]:

class EbayTrader:
    def __init__(self, auth_token: str, site_id: str = '3'):
        self.auth_token = auth_token
        self.site_id = site_id  # Default to UK (3)
        self.endpoint = 'https://api.ebay.com/ws/api.dll'
        self.compatibility_level = '1155'

    def _make_request(self, call_name: str, xml_request: str) -> Dict:
        """Make request to eBay Trading API"""
        headers = {
            'X-EBAY-API-CALL-NAME': call_name,
            'X-EBAY-API-SITEID': self.site_id,
            'X-EBAY-API-COMPATIBILITY-LEVEL': self.compatibility_level,
            'X-EBAY-API-IAF-TOKEN': self.auth_token,
            'Content-Type': 'text/xml'
        }
        
        try:
            response = requests.post(self.endpoint, headers=headers, data=xml_request)
            return xmltodict.parse(response.content)
        except Exception as e:
            print(f"Error in API call {call_name}: {str(e)}")
            return {}

    def get_category_details(self) -> Dict:
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
            <GetCategoriesRequest xmlns="urn:ebay:apis:eBLBaseComponents">
                <RequesterCredentials>
                    <eBayAuthToken>{self.auth_token}</eBayAuthToken>
                </RequesterCredentials>
                <DetailLevel>ReturnAll</DetailLevel>
                <CategorySiteID>0</CategorySiteID>
                <ViewAllNodes>true</ViewAllNodes>
            </GetCategoriesRequest>"""

        response = self._make_request('GetCategories', xml_request)
        # Debug print for category details
        if response and 'GetCategoriesResponse' in response:
            item = response['GetCategoriesResponse'].get('Categories', {})
            print("\nFull Category details :")
            print(item.get('CategoryDetails', {}))
        
        return response
    
    
    def get_item_details(self, item_id: str) -> Dict:
        """Get detailed information for a specific item"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <GetItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{self.auth_token}</eBayAuthToken>
        </RequesterCredentials>
        <ItemID>{item_id}</ItemID>
        <DetailLevel>ReturnAll</DetailLevel>
    </GetItemRequest>"""
        
        response = self._make_request('GetItem', xml_request)
        
        # Debug print for item details
        if response and 'GetItemResponse' in response:
            item = response['GetItemResponse'].get('Item', {})
            print("\nFull Shipping Details from existing item:")
            print(item.get('ShippingDetails', {}))
        
        return response

    def get_active_listings(self, page_num: int = 1, items_per_page: int = 200, include_details: bool = True) -> List[Dict]:
        """Get all active listings"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <GetMyeBaySellingRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <ActiveList>
                <Include>true</Include>
                <Pagination>
                    <EntriesPerPage>{items_per_page}</EntriesPerPage>
                    <PageNumber>{page_num}</PageNumber>
                </Pagination>
            </ActiveList>
            <DetailLevel>ReturnAll</DetailLevel>
        </GetMyeBaySellingRequest>"""
    
        response_dict = self._make_request('GetMyeBaySelling', xml_request)
        
        listings = []
        active_list = response_dict.get('GetMyeBaySellingResponse', {}).get('ActiveList', {})
        items = active_list.get('ItemArray', {}).get('Item', [])
        
        if not isinstance(items, list):
            items = [items]
            
        for item in items:
            listing = {
                'ItemID': item.get('ItemID'),
                'Title': item.get('Title'),
                'Price': item.get('SellingStatus', {}).get('CurrentPrice', {}).get('@currencyID'),
                'Amount': item.get('SellingStatus', {}).get('CurrentPrice', {}).get('#text'),
                'Quantity': item.get('Quantity'),
                'QuantityAvailable': item.get('QuantityAvailable'),
                'ListingStatus': item.get('SellingStatus', {}).get('ListingStatus'),
                'ViewItemURL': item.get('ListingDetails', {}).get('ViewItemURL'),
                'GalleryURL': item.get('PictureDetails', {}).get('GalleryURL'),
                'StartTime': item.get('ListingDetails', {}).get('StartTime'),
                'TimeLeft': item.get('TimeLeft'),
                'ShippingCost': item.get('ShippingDetails', {}).get('ShippingServiceOptions', {}).get('ShippingServiceCost', {}).get('#text')
            }
            
            # Get additional details if requested
            if include_details:
                item_details = self.get_item_details(item['ItemID'])
                item_data = item_details.get('GetItemResponse', {}).get('Item', {})
                
                listing.update({
                    'PrimaryCategoryID': item_data.get('PrimaryCategory', {}).get('CategoryID'),
                    'PrimaryCategoryName': item_data.get('PrimaryCategory', {}).get('CategoryName'),
                    'Description': item_data.get('Description'),
                    'PictureURLs': item_data.get('PictureDetails', {}).get('PictureURL', []),
                    'PaymentMethods': item_data.get('PaymentMethods', []),
                    'Location': item_data.get('Location'),
                    'Country': item_data.get('Country'),
                    'ConditionID': item_data.get('ConditionID'),
                    'ConditionDisplayName': item_data.get('ConditionDisplayName')
                })
                
            listings.append(listing)
                
        return listings

    def create_listing(self, item_details: Dict) -> Dict:
       # Create picture details XML from URLs
       picture_details = ""
       if 'photos' in item_details and item_details['photos']:
           picture_details = "<PictureDetails>"
           for photo_url in item_details['photos']:
               picture_details += f"<PictureURL>{photo_url}</PictureURL>"
           picture_details += "</PictureDetails>"

       xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <AddItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
       <RequesterCredentials>
           <eBayAuthToken>{self.auth_token}</eBayAuthToken>
       </RequesterCredentials>
       <Item>
           <Title>{item_details['Title']}</Title>
           <Description>{item_details['Description']}</Description>
           <PrimaryCategory>
               <CategoryID>{item_details['CategoryID']}</CategoryID>
           </PrimaryCategory>
           {picture_details}
           <ItemSpecifics>
               <NameValueList>
                   <Name>Brand</Name>
                   <Value>Apple</Value>
               </NameValueList>
               <NameValueList>
                   <Name>Model</Name>
                   <Value>iPhone 7</Value>
               </NameValueList>
               <NameValueList>
                   <Name>Storage Capacity</Name>
                   <Value>32 GB</Value>
               </NameValueList>
               <NameValueList>
                   <Name>Colour</Name>
                   <Value>Rose Gold</Value>
               </NameValueList>
           </ItemSpecifics>
           <StartPrice currencyID="{item_details.get('Currency', 'GBP')}">{item_details['Price']}</StartPrice>
           <Quantity>{item_details['Quantity']}</Quantity>
           <ConditionID>{item_details['ConditionID']}</ConditionID>
           <Location>{item_details.get('Location', 'London')}</Location>
           <Country>{item_details.get('Country', 'GB')}</Country>
           <Currency>{item_details.get('Currency', 'GBP')}</Currency>
           <ListingDuration>GTC</ListingDuration>
           <ListingType>FixedPriceItem</ListingType>

           <ReturnPolicy>
               <ReturnsAcceptedOption>{item_details.get('ReturnsAcceptedOption', 'ReturnsAccepted')}</ReturnsAcceptedOption>
               <ReturnsWithinOption>{item_details.get('ReturnsWithinOption', 'Days_30')}</ReturnsWithinOption>
               <ShippingCostPaidByOption>{item_details.get('ShippingCostPaidByOption', 'Buyer')}</ShippingCostPaidByOption>
           </ReturnPolicy>
           
           <ShippingDetails>
               <ShippingType>Flat</ShippingType>
               <ShippingServiceOptions>
                   <ShippingService>UK_myHermesDoorToDoorService</ShippingService>
                   <ShippingServiceCost currencyID="GBP">{item_details.get('ShippingCost', '2.99')}</ShippingServiceCost>
                   <ShippingServiceAdditionalCost currencyID="GBP">0.0</ShippingServiceAdditionalCost>
                   <ShippingServicePriority>1</ShippingServicePriority>
                   <ExpeditedService>false</ExpeditedService>
                   <ShippingTimeMin>2</ShippingTimeMin>
                   <ShippingTimeMax>3</ShippingTimeMax>
               </ShippingServiceOptions>
           </ShippingDetails>
           <DispatchTimeMax>3</DispatchTimeMax>
       </Item>
    </AddItemRequest>"""
    
       return self._make_request('AddItem', xml_request)

    def update_listing(self, item_id: str, updates: Dict) -> Dict:
        """Update an existing listing"""
        update_fields = ""
        for key, value in updates.items():
            update_fields += f"<{key}>{value}</{key}>"

        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <ReviseItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <Item>
                <ItemID>{item_id}</ItemID>
                {update_fields}
            </Item>
        </ReviseItemRequest>"""

        return self._make_request('ReviseItem', xml_request)

    def end_listing(self, item_id: str, reason: str = 'NotAvailable') -> Dict:
        """End a listing early"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <EndItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <ItemID>{item_id}</ItemID>
            <EndingReason>{reason}</EndingReason>
        </EndItemRequest>"""

        return self._make_request('EndItem', xml_request)

    def mark_as_sold(self, item_id: str) -> Dict:
        """Mark item as sold by ending listing with SoldOffeBay reason"""
        return self.end_listing(item_id, reason='SoldOffeBay')

    def get_condition_values(self, category_id: str) -> List[Dict]:
        """Get valid condition IDs and names for a specific category"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <GetCategoryFeaturesRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <CategoryID>{category_id}</CategoryID>
            <FeatureID>ConditionValues</FeatureID>
            <DetailLevel>ReturnAll</DetailLevel>
        </GetCategoryFeaturesRequest>"""

        response = self._make_request('GetCategoryFeatures', xml_request)
        category = response.get('GetCategoryFeaturesResponse', {}).get('Category', {})
        conditions = category.get('ConditionValues', {}).get('Condition', [])

        if not isinstance(conditions, list):
            conditions = [conditions]

        return [{
            'ID': condition.get('ID'),
            'DisplayName': condition.get('DisplayName'),
            'Description': condition.get('Description')
        } for condition in conditions]

    def get_shipping_services(self, country_code: str = 'GB') -> List[Dict]:
        """Get available shipping services"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <GeteBayDetailsRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <DetailName>ShippingServiceDetails</DetailName>
        </GeteBayDetailsRequest>"""
    
        response = self._make_request('GeteBayDetails', xml_request)
        services = response.get('GeteBayDetailsResponse', {}).get('ShippingServiceDetails', [])
        
        if not isinstance(services, list):
            services = [services]
        
        return [{
            'ServiceID': service.get('ShippingService'),
            'Description': service.get('Description'),
            'ServiceType': service.get('ServiceType'),
            'ValidForSellingFlow': service.get('ValidForSellingFlow') == 'true'
        } for service in services if service.get('ValidForSellingFlow') == 'true']

    def get_category_specifics(self, category_id: str) -> Dict:
        """Get required and optional item specifics for a category"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?><GetCategorySpecificsRequest xmlns="urn:ebay:apis:eBLBaseComponents"><RequesterCredentials><eBayAuthToken>{self.auth_token}</eBayAuthToken></RequesterCredentials><CategoryID>{category_id}</CategoryID></GetCategorySpecificsRequest>"""

        print("Attempting with completely flat XML...")
        print(xml_request)

        try:
            response = self._make_request('GetCategorySpecifics', xml_request)
            print("Raw API Response:")
            print(response)

            if not response:
                return {}

            category_specifics = response.get('GetCategorySpecificsResponse', {})
            print("Category Specifics Response:")
            print(category_specifics)

            return category_specifics
        except Exception as e:
            print(f"Exception details: {str(e)}")
            return {}

            
    def get_listing_requirements(self, category_id: str) -> Dict:
        """Get listing requirements for a specific category"""
        mandatory_fields = [
            'Title',          # 80 characters max
            'Description',    # HTML allowed
            'CategoryID',     # Must be a valid category ID
            'Price',         # Starting price
            'Quantity',      # Number of items
            'ConditionID',   # Must be valid for category
            'Location',      # Item location
            'Country',       # Country code (e.g., GB)
            'Currency',      # e.g., GBP
            'ListingDuration', # e.g., Days_7, GTC
            'ListingType',    # FixedPriceItem or Auction
            'PaymentMethods', # e.g., PayPal
            'ReturnPolicy',   # Must include return acceptance, period, etc.
            'ShippingDetails' # Must include at least one shipping service
        ]

        # Get category-specific requirements
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
        <GetCategoryFeaturesRequest xmlns="urn:ebay:apis:eBLBaseComponents">
            <RequesterCredentials>
                <eBayAuthToken>{self.auth_token}</eBayAuthToken>
            </RequesterCredentials>
            <CategoryID>{category_id}</CategoryID>
            <DetailLevel>ReturnAll</DetailLevel>
        </GetCategoryFeaturesRequest>"""
    
        response = self._make_request('GetCategoryFeatures', xml_request)
        category_features = response.get('GetCategoryFeaturesResponse', {}).get('Category', {})
    
        return {
            'mandatory_fields': mandatory_fields,
            'category_features': category_features,
            'listing_durations': {
                'FixedPriceItem': ['Days_3', 'Days_5', 'Days_7', 'Days_10', 'Days_30', 'GTC'],
                'Auction': ['Days_1', 'Days_3', 'Days_5', 'Days_7', 'Days_10']
            },
            'return_policy_options': {
                'returns_accepted': ['ReturnsAccepted', 'ReturnsNotAccepted'],
                'refund_options': ['MoneyBack', 'MoneyBackOrExchange', 'MoneyBackOrReplacement'],
                'returns_within': ['Days_14', 'Days_30', 'Days_60'],
                'shipping_cost_paid_by': ['Buyer', 'Seller']
            }
        }

    def upload_photo(self, image_path: str) -> Optional[str]:
        """Upload a photo to eBay and return the URL"""
        xml_request = f"""<?xml version="1.0" encoding="utf-8"?>
    <UploadSiteHostedPicturesRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{self.auth_token}</eBayAuthToken>
        </RequesterCredentials>
        <PictureName>{os.path.basename(image_path)}</PictureName>
    </UploadSiteHostedPicturesRequest>"""
    
        try:
            with open(image_path, 'rb') as img_file:
                files = {
                    'XML Payload': ('', xml_request, 'text/xml'),
                    'image': (os.path.basename(image_path), img_file, 'application/octet-stream')
                }
                
                response = requests.post(
                    'https://api.ebay.com/ws/api.dll',
                    files=files,
                    headers={
                        'X-EBAY-API-SITEID': self.site_id,
                        'X-EBAY-API-COMPATIBILITY-LEVEL': self.compatibility_level,
                        'X-EBAY-API-IAF-TOKEN': self.auth_token,
                        'X-EBAY-API-CALL-NAME': 'UploadSiteHostedPictures'
                    }
                )
                
                response_dict = xmltodict.parse(response.content)
                return response_dict.get('UploadSiteHostedPicturesResponse', {}).get('SiteHostedPictureDetails', {}).get('FullURL')
        except Exception as e:
            print(f"Error uploading image: {str(e)}")
            return None


    def get_end_reasons(self) -> List[str]:
        """Get valid ending reason codes"""
        xml_request = """<?xml version="1.0" encoding="utf-8"?>
    <GetItemRequest xmlns="urn:ebay:apis:eBLBaseComponents">
        <RequesterCredentials>
            <eBayAuthToken>{}</eBayAuthToken>
        </RequesterCredentials>
        <DetailLevel>ReturnAll</DetailLevel>
        <OutputSelector>EndItemResponseDetails</OutputSelector>
    </GetItemRequest>""".format(self.auth_token)

        response = self._make_request('GetItem', xml_request)
        print("Raw Response:", response)  # Let's see what we get back

    # Let's also hard-code the known working values for now
    known_reasons = [
        {'code': 'NotAvailable', 'description': 'Item is no longer available'},
        {'code': 'LostOrBroken', 'description': 'Item is lost or broken'},
        {'code': 'Incorrect', 'description': 'Listing contains errors'},
        {'code': 'CustomCode', 'description': 'Custom reason'}
    ]

    print("\nKnown working end reasons:")
    for reason in known_reasons:
        print(f"Code: {reason['code']}, Description: {reason['description']}")

if __name__ == "__main__":
    # Initialize the trader
    trader = EbayTrader(auth_token=auth_token)

    # Get all active listings
    listings = trader.get_active_listings()


    
    # # Create a new listing
    # new_item = {
    #     'Title': 'Test Item',
    #     'Description': 'This is a test item',
    #     'CategoryID': '11450',  # You'll need the correct category ID
    #     'Price': '29.99',
    #     'Quantity': '1',
    #     'PayPalEmail': 'your.paypal@email.com',
    #     'Location': 'London',
    #     'ShippingCost': '3.99'
    # }
    # create_result = trader.create_listing(new_item)

    # # Update a listing
    # updates = {
    #     'Title': 'Updated Title',
    #     'Price': '39.99'
    # }
    # update_result = trader.update_listing('ITEM_ID', updates)

    # # End a listing
    # end_result = trader.end_listing('ITEM_ID')

    # # Mark as sold elsewhere
    # sold_result = trader.mark_as_sold('ITEM_ID')

listings


Known working end reasons:
Code: NotAvailable, Description: Item is no longer available
Code: LostOrBroken, Description: Item is lost or broken
Code: Incorrect, Description: Listing contains errors
Code: CustomCode, Description: Custom reason

Full Shipping Details from existing item:
{'ApplyShippingDiscount': 'false', 'CalculatedShippingRate': {'WeightMajor': {'@measurementSystem': 'Metric', '@unit': 'kg', '#text': '0'}, 'WeightMinor': {'@measurementSystem': 'Metric', '@unit': 'gm', '#text': '0'}}, 'SalesTax': {'SalesTaxPercent': '0.0', 'ShippingIncludedInTax': 'false'}, 'ShippingServiceOptions': [{'ShippingService': 'UK_myHermesDoorToDoorService', 'ShippingServiceCost': {'@currencyID': 'GBP', '#text': '2.99'}, 'ShippingServicePriority': '1', 'ExpeditedService': 'false', 'ShippingTimeMin': '2', 'ShippingTimeMax': '3'}, {'ShippingService': 'UK_CollectInPerson', 'ShippingServiceCost': {'@currencyID': 'GBP', '#text': '0.0'}, 'ShippingServicePriority': '2', 'ExpeditedService': 'false'}],

[{'ItemID': '167279800027',
  'Title': 'Apple iPhone 7 - 32GB - Rose Gold (BT Mobile) A1778 (GSM)',
  'Price': 'GBP',
  'Amount': '469.99',
  'Quantity': '1',
  'QuantityAvailable': '1',
  'ListingStatus': None,
  'ViewItemURL': 'https://www.ebay.co.uk/itm/Apple-iPhone-7-32GB-Rose-Gold-BT-Mobile-A1778-GSM--/167279800027',
  'GalleryURL': 'https://i.ebayimg.com/images/g/0NIAAOSwMW9nmKxM/s-l140.jpg',
  'StartTime': '2025-01-28T10:07:43.000Z',
  'TimeLeft': 'P2DT11H44M55S',
  'ShippingCost': '2.99',
  'PrimaryCategoryID': '9355',
  'PrimaryCategoryName': 'Mobile Phones & Communication:Mobile & Smart Phones',
  'Description': None,
  'PictureURLs': 'https://i.ebayimg.com/00/s/MTYwMFgxMjAx/z/0NIAAOSwMW9nmKxM/$_57.JPG?set_id=880000500F',
  'PaymentMethods': [],
  'Location': 'Leeds',
  'Country': 'GB',
  'ConditionID': '3000',
  'ConditionDisplayName': 'Used'}]

In [48]:
# Usage:
# valid_reasons = trader.get_end_reasons()
# print("Valid ending reasons:")
# print(valid_reasons)
# for reason in valid_reasons:
#     print(f"Code: {reason['code']}, Description: {reason['description']}")

categories = trader.get_category_details()
print(type(categories['GetCategoriesResponse']))
categories['GetCategoriesResponse'].keys()
#



Full Category details :
{}
<class 'dict'>


dict_keys(['@xmlns', 'Timestamp', 'Ack', 'Version', 'Build', 'CategoryArray', 'CategoryCount', 'UpdateTime', 'CategoryVersion', 'ReservePriceAllowed', 'MinimumReservePrice'])

In [57]:
for n in range(0, 100):
    print(categories['GetCategoriesResponse']['CategoryArray']['Category'][n]['CategoryName'])

Antiques
Architectural & Garden
Balusters
Barn Doors & Barn Door Hardware
Beams
Ceiling Tins
Chandeliers, Sconces & Lighting Fixtures
Columns & Posts
Corbels
Doors
Finials
Fireplaces, Mantels & Fireplace Accessories
Garden
Hardware
Door Bells & Knockers
Door Knobs & Handles
Door Plates & Backplates
Drawer Pulls
Escutcheons & Key Hole Covers
Heating Grates & Vents
Hinges
Hooks, Brackets & Curtain Rods
Locks, Latches & Keys
Nails
Screws
Switch Plates & Outlet Covers
Other Antique Hardware
Pediments
Plumbing & Fixtures
Signs & Plaques
Stained Glass Windows
Stair & Carpet Rods
Tiles
Weathervanes & Lightning Rods
Windows, Shutters & Sash Locks
Reproductions
Price Guides & Publications
Other Architectural Antiques
Asian Antiques
Burma
China
Amulets
Armor
Baskets
Bells
Bowls
Boxes
Bracelets
Brush Pots
Brush Washers
Cabinets
Chairs
Chests
Fans
Figurines & Statues
Glasses & Cups
Incense Burners
Ink Stones
Masks
Necklaces & Pendants
Paintings & Scrolls
Plates
Pots
Rings
Robes & Textiles
Seals
Sn

In [ ]:
# requirements = trader.get_listing_requirements(category_id)
# print("\nMandatory fields:", requirements['mandatory_fields'])

# specifics = trader.get_category_specifics(category_id)
# print("\nRequired item specifics:", specifics)

# Example usage with photos
new_listing = {
    'Title': 'Apple iPhone 7 - 32GB - Graphite',
    'Description': 'Used iPhone 7 in good condition',
    'CategoryID': '9355',
    'Price': '329.99',
    'Quantity': '1',
    'ConditionID': '3000',  # Used
    'Location': 'London',
    'Currency': 'GBP',
    'ShippingCost': '2.99',
    'photos': ['https://imgs.search.brave.com/kjobBP0SIaMgF_mQ-FEpVrHtPaPYLdfjXr8K74Q1Q8k/rs:fit:860:0:0:0/g:ce/aHR0cHM6Ly90ZWNo/cHJvcy5jby51ay9j/ZG4vc2hvcC9maWxl/cy84YTg4OWRlZi1j/NDBlLTQ2ZjYtYmRk/OS0zMDA3YWZhMTZh/NTMtM18wZGUxYmIy/OS0yYWI3LTQ1ZmMt/YjhlOC1iNDBjMDJk/YTg3MTkuanBnP3Y9/MTcxNjM2MzI2MyZ3/aWR0aD0xMDI0', 
               'https://cdn.idealo.com/folder/Product/5113/0/5113043/s3_produktbild_gross/apple-iphone-7-128gb-rose-gold.jpg']  # Need to provide actual photo URLs
}


result = trader.create_listing(new_listing)
result

In [18]:
# Get basic listing info only
basic_listings = trader.get_active_listings(include_details=False)

# Get full details including categories
detailed_listings = trader.get_active_listings(include_details=True)

detailed_listings


Full Shipping Details from existing item:
{'ApplyShippingDiscount': 'false', 'CalculatedShippingRate': {'WeightMajor': {'@measurementSystem': 'Metric', '@unit': 'kg', '#text': '0'}, 'WeightMinor': {'@measurementSystem': 'Metric', '@unit': 'gm', '#text': '0'}}, 'SalesTax': {'SalesTaxPercent': '0.0', 'ShippingIncludedInTax': 'false'}, 'ShippingServiceOptions': [{'ShippingService': 'UK_myHermesDoorToDoorService', 'ShippingServiceCost': {'@currencyID': 'GBP', '#text': '2.99'}, 'ShippingServicePriority': '1', 'ExpeditedService': 'false', 'ShippingTimeMin': '2', 'ShippingTimeMax': '3'}, {'ShippingService': 'UK_CollectInPerson', 'ShippingServiceCost': {'@currencyID': 'GBP', '#text': '0.0'}, 'ShippingServicePriority': '2', 'ExpeditedService': 'false'}], 'ShippingType': 'Flat', 'ThirdPartyCheckout': 'false', 'ShippingDiscountProfileID': '0', 'InternationalShippingDiscountProfileID': '0', 'SellerExcludeShipToLocationsPreference': 'true'}


[{'ItemID': '167279800027',
  'Title': 'Apple iPhone 7 - 32GB - Rose Gold (BT Mobile) A1778 (GSM)',
  'Price': 'GBP',
  'Amount': '449.99',
  'Quantity': '1',
  'QuantityAvailable': '1',
  'ListingStatus': None,
  'ViewItemURL': 'https://www.ebay.co.uk/itm/Apple-iPhone-7-32GB-Rose-Gold-BT-Mobile-A1778-GSM--/167279800027',
  'GalleryURL': 'https://i.ebayimg.com/images/g/0NIAAOSwMW9nmKxM/s-l140.jpg',
  'StartTime': '2025-01-28T10:07:43.000Z',
  'TimeLeft': 'P2DT12H6M36S',
  'ShippingCost': '2.99',
  'PrimaryCategoryID': '9355',
  'PrimaryCategoryName': 'Mobile Phones & Communication:Mobile & Smart Phones',
  'Description': None,
  'PictureURLs': 'https://i.ebayimg.com/00/s/MTYwMFgxMjAx/z/0NIAAOSwMW9nmKxM/$_57.JPG?set_id=880000500F',
  'PaymentMethods': [],
  'Location': 'Leeds',
  'Country': 'GB',
  'ConditionID': '3000',
  'ConditionDisplayName': 'Used'}]

In [ ]:
result = trader.end_listing(item_id="167281756063", reason="NotAvailable")
result

In [19]:
# Let's say you want to list in the Mobile Phones category
category_id = '9355'  # Mobile Phones category

# Get all requirements and options
conditions = trader.get_condition_values(category_id)
print("Available conditions:")
for condition in conditions:
    print(f"ID: {condition['ID']} - {condition['DisplayName']}")

Available conditions:
ID: 1000 - New
ID: 1500 - Opened – never used
ID: 3000 - Used
ID: 7000 - For parts or not working


In [ ]:
shipping_services = trader.get_shipping_services()
# print("\nAvailable shipping services:")
# for service in shipping_services:
#     print(f"ID: {service['ServiceID']} - {service['Description']}")

In [ ]:
# Let's try it with your existing item ID from earlier
item_id = "167279800027"  # Your successful listing ID
details = trader.get_item_details(item_id)

In [20]:
# THINK THIS IS REDUNDANT CODE BUT CHECK CLASS AS MIGHT BE USEFUL



class EbayTokenManager:
    def __init__(self, config_file='ebay_config.yaml'):
        self.config_file = config_file
        self.config = self._load_config()

    def _load_config(self):
        """Load configuration from YAML file or create with defaults"""
        if os.path.exists(self.config_file):
            with open(self.config_file, 'r') as file:
                return yaml.safe_load(file)
        return {
            'credentials': {
                'client_id': None,
                'client_secret': None,
                'ru_name': None,
            },
            'tokens': {
                'refresh_token': None,
                'created_date': None,
                'access_token': None
            }
        }

    def _save_config(self):
        """Save configuration to YAML file"""
        with open(self.config_file, 'w') as file:
            yaml.dump(self.config, file)

    def setup_credentials(self, client_id, client_secret, ru_name):
        """Store eBay API credentials"""
        self.config['credentials']['client_id'] = client_id
        self.config['credentials']['client_secret'] = client_secret
        self.config['credentials']['ru_name'] = ru_name
        self._save_config()

    def store_refresh_token(self, refresh_token):
        """Store new refresh token with current date"""
        self.config['tokens']['refresh_token'] = refresh_token
        self.config['tokens']['created_date'] = datetime.now().isoformat()
        self._save_config()

    def should_refresh_token(self):
        """Check if refresh token is older than 1 year"""
        if not self.config['tokens']['created_date']:
            return True
            
        created_date = datetime.fromisoformat(self.config['tokens']['created_date'])
        one_year_old = datetime.now() - timedelta(days=365)
        return created_date < one_year_old

    def get_access_token(self):
        """Get new access token using refresh token"""
        if not self.config['tokens']['refresh_token']:
            raise Exception("No refresh token available. Please generate one first.")

        if not self.config['credentials']['client_id'] or not self.config['credentials']['client_secret'] or not self.config['credentials']['ru_name']:
            raise Exception("API credentials not set. Please run setup_credentials() first.")
            
        url = 'https://api.ebay.com/identity/v1/oauth/token'
        
        credentials = f"{self.config['credentials']['client_id']}:{self.config['credentials']['client_secret']}::{self.config['credentials']['ru_name']}"
        encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
        
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Authorization': f'Basic {encoded_credentials}'
        }
        
        encoded_refresh_token = quote(self.config['tokens']['refresh_token'].strip())
        data = f"grant_type=refresh_token&refresh_token={encoded_refresh_token}"

        response = requests.post(url, headers=headers, data=data)

        if response.status_code == 200:
            access_token = response.json()['access_token']
            self.config['tokens']['access_token'] = access_token
            self._save_config()
            return access_token
        else:
            raise Exception(f"Failed to get access token. Status: {response.status_code}")

def main():

    manager = EbayTokenManager()

    # First time setup (only need to do this once):
    if not manager.config['credentials']['client_id']:
        manager.setup_credentials(
            client_id="TomArund-ReadInve-PRD-e66f79d08-8f8acc4d",
            client_secret="PRD-66f79d08e8ad-bf57-4662-affd-8697",
            ru_name = "Tom_Arundel-TomArund-ReadIn-zrmym"
        )
        manager.store_refresh_token('YOUR_INITIAL_REFRESH_TOKEN')

    # Regular usage
    try:
        if manager.should_refresh_token():
            print("WARNING: Refresh token is older than 1 year or doesn't exist!")
            print("Please generate a new refresh token and use store_refresh_token()")

        access_token = manager.get_access_token()
        print("Successfully obtained access token")
        return access_token

    except Exception as e:
        print(f"Error: {e}")
        return None

if __name__ == "__main__":
    main()


"""
The YAML file (ebay_config.yaml) will now look like this:

yamlCopycredentials:
  client_id: 'YOUR_CLIENT_ID'
  client_secret: 'YOUR_CLIENT_SECRET'
tokens:
  refresh_token: 'v^1.1#i^1#...'
  created_date: '2025-01-28T14:48:06.123456'
  access_token: 'v^1.1#i^1#...'

To use it:

First time setup:

pythonCopymanager = EbayTokenManager()
manager.setup_credentials(
    client_id='your_client_id',
    client_secret='your_client_secret'
)
manager.store_refresh_token('your_refresh_token')

Regular usage:

pythonCopymanager = EbayTokenManager()
access_token = manager.get_access_token()
The benefits of this approach:

All credentials and tokens in one place
Only need to set up credentials once
Clear separation between credentials and tokens
Easy to backup and version control (though make sure to .gitignore the yaml file!)
"""

NameError: name 'yaml' is not defined

In [ ]:
headers = {
    "X-EBAY-API-SITEID" :3,
    "X-EBAY-API-COMPATIBILITY-LEVEL" :967,
    "X-EBAY-API-CALL-NAME" : "GetMyeBaySelling", 
    "X-EBAY-API-IAF-TOKEN": "v^1.1#i^1#r^0#p^3#f^0#I^3#t^H4sIAAAAAOVafWwbZxmP89HRdlk3wUZVVZPnUXUjOvu989idEXtz4kviJFAAAAAAA5c22nTSJv13t17ziX34d69l8ThQ1mGhjppU0UFSN0fpDCBhpA2Om1oYqvYVD6EtMGQitD+AMTH6KAgBpSPqtJ4z05TN9AmscM4CStSfO89X7/nfZ7nfe45g6UdOz/62NBjf+/23dK+sgSW2n0+ejfYuaOr57aO9n1dbaCBwLey9JGlzuWOC7021LUKn0d2xTRs5F/QNcPma4uJgGMZvAlt1eYNqCObxxJfSGVHeSYI+IplYlMytYA/k04EZElhOSnGIBEyIheWyapxVWbRTAQURIejYRpxCku+KOS2bTsoY9gYGjgRYAATpQBNMWyRpnk6ztNMkAXMVMB/GFm2ahqEJAgCyZq1fI3XajD15pZC20YWJkICyUxqoDCeyqSFsWJvqEFWctUNBQyxY19/1W/KyH8Yag66uRq7Rs0XHElCth0IJesarhfKp64a04T5NU9HGBkwEIoRBcVkuE2uHDAtHeKb2+GuqDKl1Eh5ZGAVVzfyKPGGOIMkvHo1RkRk0n733yEHaqqiIisREPpSRycKQj7gL+Ryljmnykh2kdJcLE7+WCaQxKauV0vYNDUdGqt66sJWvbxOUb9pyKrrM9s/ZuI+RIxG610DGlxDiMaNcSulYNegRrrwqguJJVPuntY30cHThrutSCd+8NcuN96AqxFxLQa2KyZQJMpFJS4mioiJIlH+DzHh5vqW4yLpbk0qlwu5tpDUrlI6tGYRrmhQQpRE3OvoyFJlnqQ0E2YVRMkxTqEinKJQYlSOUbSCEEBIFCWO/T8KD4wtVXQwWguR9TdqGBOBgmRWUM7UVKkaWE9SqzirAbFgJwLTGFf4UGh+fj44Hw6aVjnEAECHJrOjBWka6TCwRqtuTEyptdCQEOGyVR5XK8SaBRJ5RLlRDiTDlpyDFq4WkKaRhatxe51tyfWrNwDZr6nEA0WiwlsYh0wbI7klaDKaUyVUUmVvIXNznWHCcZZmQQQAwLYEUjPLqpFFeNr0GEy3KGTSLWEjNRRib6FqqC4gslqF4gxHkW8AtAQWli1Uq0b1hsRbsFP9/UKuKLS2nalKJaPrDoaihjIei9Yo6Xtj4e3ZQGG1+SJkbq57BqN7VJf6x7NZId8vlCZGSqPF1ja04jheK64T6VSfIFgwz+gtQXPbKV6FCo/NWWR473jMCwN5oTBUKo6PCGMtIc0jxUL2dNHF6bWsTB1KjaTIJyv0QNPsG8nbjANGJ2AeO4cnI3NSenJxYXFqkc4p5TklvMhFJueyBTQ1Y8QksWcsp7FCZjqLh2er84lES04qIMlCGxxFbq6/3w6a7ekZnJzLlefFYWYgfuiYFnV0cZKNHo2O55W0rYf6JxaGY8wRR5pvzQHZstcyfftaqKI3U9yqJ2apVoFK5KolkELZc7XaHUlFEBOjuQiAMSCFRRaJSGEVReEYEdAtNxv/PbxurjeFuWjqKcsxZCqPoJwx5hCVy6cpFIspcU4GLMUqLJSkSGsPPhXPbfV2HctrPZZ76S2IudTRrDBWLDAlUHKffUqpwbwgZK9N3ppDbLtDCG8hdfltIgBW1KDbJwUlUw+Z0MHT7lKpZrF/M0QhG2lasD60IpKDFskJ09CqzTBvgUclaWdg06puTqGb6zcQsAWlJKtNx8DNYFxl3QKH4miKqmluojSjsIF9K2YaUKtiVbKbUqkabsTZW2CpwGoNoKzaFTdfNsVJ1nRkSSioyvUZeTPGWogohLWRcDNMW1S5ZrJhYlVRpboM2xFtyVIrm7diYznN+MImebClbaszbEpVAxeSkabOIava2kTQ1FVJ1W5cT91c/5/U1MG+1iZlSFYtJOGSY6neOixIw1OqdTxIo9Y1P9SipVdb7AdImHhx+JlLFQpHxvOtjVfSaM5rD+QcghGRi4QpwDA0FQlHEcWykKNEBkYRC8MAxFt7FnsfB76dj5zcHGg6HmZBLAzCmx7wrltoeNH0b68YQ9e/4U+21T70su81sOw72+7zgV5wgL4X3LOjY6Kz49Z9torJ4QWVoK2WDYgdCwVnUbUCVav9g20/vG1UfmRo9NKS6HzzyF8fYNu6G35gsPIQ2Lv2E4OdHfTuht8bgP3X7nTRez7czUQBzbA0TcdpZgrce+1uJ31X54fOn/nWWxd2/+3FrmetttuPf3n2vT/sfxd0rxH5fF1tncu+tk+9efc7P7hl4MQ78c/85eeXz8R+fecn7nrm5fPUE5+PCa9H73jy3cLAiU9+9o37p2eqz9ts7xvHOi+ffuXAPy5/ffClL7UfGPrq13pKPStTnz7355Bx/2DygbO5RM+p35vD3xBO7T1x4vHOhx8dea735I9++s9bxe917Tr3q+fSM5e4t5/15/Clp5bA/pGvXHzod9Gn+34zv+OLu648Pay/1z71syuvstKjpy5+J7mnOyI+uGvlvmeeP9/x4Nu+8+bpjxXfevH21+YvXui7/NsrmadeOLP3u0/cfc/w0Me/XXkdHvzx0V0HF6F1ah/3y3z4zrPLXziYGpeOdf/p+12f+8meh5kP7H+849wl8aWTx+87/sfTr87Qk794k6vv5b8AJjStGPohAAA="
}

ebay_cwd = "/Users/wommy/Documents/GitHub/PROJECTS/HANKS/inventory_system/app/services/ebay/ebaysdk-python"

# print(os.getcwd())
if os.getcwd() != ebay_cwd:
    os.chdir(ebay_cwd)

try:
    api = Connection(appid='YABADABADOOOO', config_file='ebay.yaml')
    response = api.execute('GetStore', {'keywords': 'iphone'})

    assert(response.reply.ack == 'Success')
    assert(type(response.reply.timestamp) == datetime.datetime)
    assert(type(response.reply.searchResult.item) == list)

    item = response.reply.searchResult.item[0]
    assert(type(item.listingInfo.endTime) == datetime.datetime)
    assert(type(response.dict()) == dict)

except ConnectionError as e:
    print(e)
    print(e.response.dict())

